In [1]:
import numpy as np
import pandas as pd
import statsmodels.stats.power as smp

# 1. Before Loading any Data

## A. How large a sample size would be needed to detect an increase in click throughs equal to 10% of the mean, with 90% power and 5% significance? Assume the mean click through rate is 5%.

According to the power calculation formula,
$$
\text{power} = F(\alpha = 0.05, N, \text{standard effect size}=0.05*0.1/sqrt(0.05*(1-0.05))) 
$$

In [2]:
sd = np.sqrt( .05*(1-.05))
std_effect_size = .05*.1 / sd
sample_n = smp.tt_ind_solve_power(effect_size=std_effect_size, power =.9, alpha=.05)

print(int(sample_n))

39929


## B. If the typical treatment arm has 3200 observations, what is the minimum detectable effect (not standardized), given 5% significance ?

In [3]:
mde_std = smp.tt_ind_solve_power(nobs1 = 3200, power = 0.9, alpha=0.05)
mde = mde_std * sd
print(mde)

0.017664008734157317


## C. If the typical treatment arm has 3200 observations, what is our power to detect an effect equal to 10% of the mean, with 5% significance?

In [4]:
sd = np.sqrt( .05*(1-.05))
std_effect_size = .05*.1 / sd
power = smp.tt_ind_solve_power(nobs1 = 3200, alpha=.05,
                                         effect_size = std_effect_size)
print(power)

0.1506084136476183


## D. Your organization has a website which gets 4,000 views/hour. You are considering running an A/B test of headlines to determine headline effectiveness. For how long would you have to run the test to be able to detect an increase in the click through rate from 5% to 5.5% (with 90% power and 5% significance)?  ("A/B test" means you have 1 control arm and 1 treatment arm.)

In [5]:
obs = smp.tt_ind_solve_power(power=.9, alpha=.05,
                                         effect_size = (0.055-0.05)/sd )
obs/(4000/2)

19.96457760713906

It takes 19.96 hours to run the test to be able to detect an increase in the click though rate from 5% to 5.5%, with 90% power and 5% significance.

# 2. Load Data

In [6]:
data = pd.read_csv('upworthy_questions.csv')
data.head()

clickability_test_id  impressions0  impressions1  clicks0  clicks1
0  51436060220cb800020001e3          2707          2606       50       42
1  51436061220cb800020001e8          2702          2720       21       14
2  51436061220cb800020001ec          2817          2945       22       29
3  51436063220cb800020002a7          5268          5414      112      120
4  51436063220cb800020002cc          5293          5377       58       74

In [7]:
# Calculate the total number of impression and total number of clicks in each arm of the experiment, and then calculate the overall clickthrough rate in each arm.
tot_n_imp0 = data['impressions0'].sum()
tot_n_imp1 = data['impressions1'].sum()
print("Total number of impression in no-question-mark group:", tot_n_imp0)
print("Total number of impression in question-mark group:", tot_n_imp1)
print("\n")

tot_n_click0 = data['clicks0'].sum()
tot_n_click1 = data['clicks1'].sum()
print("Total number of clicks in no-question-mark group:", tot_n_click0)
print("Total number of impression in question-mark group:", tot_n_click1)
print("\n")

oa_click_rate0 = tot_n_click0/tot_n_imp0
oa_click_rate1 = tot_n_click1/tot_n_imp1
print("Overall clickthrough rate in no-question-mark group:", oa_click_rate0)
print("Overall clickthrough rate in question-mark group:", oa_click_rate1)

Total number of impression in no-question-mark group: 24870155
Total number of impression in question-mark group: 24867195


Total number of clicks in no-question-mark group: 302207
Total number of impression in question-mark group: 273798


Overall clickthrough rate in no-question-mark group: 0.012151391899246305
Overall clickthrough rate in question-mark group: 0.011010409497331725


In [8]:
## What is the difference in means?
diff = oa_click_rate1 - oa_click_rate0
diff

-0.0011409824019145802

In [9]:
## Form a 95% confidence interval for the difference in means. Is the difference statistically significant?
se = np.sqrt(oa_click_rate0*(1-oa_click_rate0)/tot_n_imp0+oa_click_rate1*(1-oa_click_rate1)/tot_n_imp1)
lb = diff-1.96*se
ub = diff+1.96*se
ci = [lb, ub]
ci

[-0.0012004499012693817, -0.0010815149025597787]

It is statistically significant.

In [10]:
# Now treat each experiment as a separate observation. For each experiment, calculate the effect of question mark on the outcome. This will give you 6,939 ATEs.
ate_table = data.copy()
ate_table['click_rate0'] = data['clicks0']/data['impressions0']
ate_table['click_rate1'] = data['clicks1']/data['impressions1']
ate_table['ate'] = ate_table['click_rate1'] - ate_table['click_rate0']
ate_table.head()

clickability_test_id  impressions0  impressions1  clicks0  clicks1  \
0  51436060220cb800020001e3          2707          2606       50       42   
1  51436061220cb800020001e8          2702          2720       21       14   
2  51436061220cb800020001ec          2817          2945       22       29   
3  51436063220cb800020002a7          5268          5414      112      120   
4  51436063220cb800020002cc          5293          5377       58       74   

   click_rate0  click_rate1       ate  
0     0.018471     0.016117 -0.002354  
1     0.007772     0.005147 -0.002625  
2     0.007810     0.009847  0.002037  
3     0.021260     0.022165  0.000904  
4     0.010958     0.013762  0.002804

In [11]:
## What is the average ATE, averaging over all the experiments?
avg_ate = ate_table['ate'].mean()
avg_ate

-0.0012118772185663103

In [12]:
## How many of the ATEs are statistically significant? (hint: consider making a confidence interval for each experiment)
ate_table['sem0'] = np.sqrt(ate_table['click_rate0']*(1-ate_table['click_rate0'])/ate_table['impressions0'])
ate_table['sem1'] = np.sqrt(ate_table['click_rate1']*(1-ate_table['click_rate1'])/ate_table['impressions1'])
ate_table['se'] = np.sqrt(ate_table['sem0']**2+ate_table['sem1']**2)
ate_table['lb'] = ate_table['ate'] - 1.96*ate_table['se']
ate_table['ub'] = ate_table['ate'] + 1.96*ate_table['se']
# ate_table['sign'] = ate_table['lb']*ate_table['ub']
subdata = ate_table[(ate_table['ub']<0) | (ate_table['lb']>0)]
subdata['ate'].count()

2315

In [13]:
## What is the mean ATE among experiments where the ATE is statistically significant?
subdata['ate'].mean()

-0.0030497218997479945

## Why is the answer in (c) different than in (b)?

The difference between the mean ATE from all experiments (answer in (a)) and the mean ATE from only the statistically significant experiments (answer in (c)) is due to what is known as selection bias. 

## If you were an organization that ran many experiments and only paid attention to the significant ones, would your answers be reliable? Explain briefly.

No. Focusing only on significant subgroup would be introducing selection bias. Selection bias here affects the reliability of findings because it skews the data pool toward false positives, undermining the accuracy of generalizations. Without including non-significant results, there’s no control for random variation, making it impossible to determine if outcomes are consistent in real life experiments. This is also called a “file drawer effect,” where only positive findings are reported, presenting an incomplete view of the interventions’ real-world effectiveness.

# Develop a hypothesis about what features of headline generate higher click through rates. Think about an experiment that would let you test it, using the Upworthy experiment database. The key challenge in testing your hypothesis is whether there are experiments that A/B test the feature, and whether you can identify those experiments. For example, we can tell if there is a question in a headline because the data have a headline column, and we can see if it contains a ‘?’. Make a pre-analysis plan at aspredicted.org, and turn in your pre-analysis plan with your write-up.

Hypothesis: Headlines written in all uppercase letters will generate a higher click-through rate compared to normally formed headlines.

Dependent variable: click-through rate. We'll collect the impressions and clicks and generate the click-through rate form those 2 variables.

Participants will be assigned to 2 conditions: One with all-uppercase title, the other with normally formatted title.

I'll calculate the ate on click-through rate and its confidence interval to see if the uppercase headlines have significant effects on the clicks.

Just a draft, any reasonable effort will be awardeed a full mark here.